In [1]:
%matplotlib inline

import arcgis
import csv
import io
import json
import os
import PIL
import requests
import datetime
import time
import pymongo

import cognitive_face as CF
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from arcgis import geometry
from arcgis.gis import GIS
from copy import deepcopy
from bson import json_util
from IPython.display import HTML
from io import BytesIO
from matplotlib import patches
from PIL import Image

print (PIL.PILLOW_VERSION)


5.2.0


In [2]:
gis = GIS('http://enterprisegis.natgov.geocloud.com/portal', 'james_jones', 'QWerty654321@!')

In [3]:
client = pymongo.MongoClient('mongodb://localhost:27017/')
db  = client['KBI']
faces_collection = db['faces']

In [4]:
# Sets the subscription key for the Microsoft Cognitive Services Faces API
subscription_key = '53242be6635c420c807d15a44a7015cf'
assert subscription_key

# Sets the Base URL that will be passed to the 
base_url = 'https://eastus.api.cognitive.microsoft.com/face/v1.0/'

# Passes the subscription key and the Base URL to the Cognitive Faces SDK
CF.Key.set(subscription_key)
CF.BaseUrl.set(base_url)

In [5]:
# Sets the base URL for the Detect Faces tool from
face_api_url = base_url + 'detect'

# Sets the base URL for POSTing to GeoEvent
geoevent_url = 	r'http://wdcsol0000046.esri.com:6180/geoevent/rest/receiver/KBI'

In [6]:
# Headers and Parameters for the Faces API Requests Call
mcs_headers = { 'Ocp-Apim-Subscription-Key': subscription_key }
    
mcs_params = {
    'returnFaceId': 'true',
    'returnFaceLandmarks': 'false',
    'returnFaceAttributes': 'age,gender,headPose,smile,facialHair,glasses,emotion,hair,makeup,occlusion,accessories,blur,exposure,noise',
}

geoevent_headers = {
    'Content-Type': 'application/json',
}

In [7]:
# Location of source images
img_Locs = r'C:\xampp1\htdocs\camera\streamMobile\uploads'

# Output location for marked up images
markUp = r'C:\xampp1\htdocs\camera\Image_MarkUp'

# Base web end point for the hosted images
baseWeb = 'http://40.76.87.212/camera/'

# Hosted location for the marked up images
web_markUp = baseWeb + 'Image_MarkUp/'

# Hosted location for the source images
web_Source = baseWeb + 'streamMobile/uploads/'

# Location of output JSON Files
responseFP = r'C:\Users\dif_user\Documents\Cognitive-Facial-Recognition\JSON'

# Location of GeoEvent JSON REST endpoint
geoevent_json_path = r'C:\Data\faces\json'
geoevent_url = r'http://wdcrealtimeevents:6180/geoevent/rest/receiver/KBI-SOFwerx-Faces'


In [8]:
# Known Bad Guy source image and biograhical info
#knownBadGuy = 'http://40.76.87.212/camera/source_images/file-2.jpeg'
#bgName = 'Target Individual'
delayTime = 1

In [9]:
# Obtains the necessary feature classes
camera_points = gis.content.get('8c2c8ed3465141c7aa70ac5d85d4896f')
cameras_lyr = camera_points.layers[0]

In [10]:
# Helper lists and dicts used during processing
faceList = []
allFaces = []
done_urls = []
faceID = []
da_Faces = []
facePics = {}
addedRows = []
addedDaFaces = []

In [11]:
def process_image(image_file):
    img_path = os.path.join(img_Locs, image_file)
    im = Image.open(img_path)
    width, height = im.size
    if img.split("_")[0] == 'EsriUC':
        pass
    else:
        if width > height:
            im1 = im.rotate(-90)
            im1.save(img_path)

    image_url = web_Source + "/" + image_file

    # Allows the computer to read the image and process it
    image_file = BytesIO(requests.get(image_url).content)
    image = Image.open(image_file)
        
    return image, image_url

In [12]:
def get_spatial_info(csv_file_path):
    with open(csv_file_path) as csvfile:
        readCSV = csv.reader(csvfile)
        for row in readCSV:
            sRow = row[0].split('|')
            print(sRow)
            feature_name = sRow[0]
            latitude = sRow[1]
            longitude = sRow[2]
            feature_time = sRow[3]
                
    return feature_name, latitude, longitude, feature_time

In [13]:
def update_json(input_json_file, image_name, image_url, latitude, longitude, face_count):
    input_json_file['image_name'] = image_name
    input_json_file['image_url'] = image_url
    input_json_file['lat'] =latitude
    input_json_file['long'] = longitude
    input_json_file['number_faces'] = face_count

In [14]:
def update_fc(gis_connection, layer, latitude, longitude, image_name, image_url, face_count):
    layer_fset = layer.query(return_geometry=True)
    all_features = layer_fset.features
    original_features = all_features[0]
    template_feature = deepcopy(original_features)
    features_for_update = []
    new_feature = deepcopy(template_feature)
    
    input_geometry = {'y':latitude,
                      'x':longitude}
    
    output_geometry = geometry.project(geometries = [input_geometry],
                                                    in_sr=4326,
                                                    out_sr=4326,
                                                    gis=gis)

    new_feature.geometry = output_geometry[0]
    
    new_feature.attributes['image_name'] = image_name
    new_feature.attributes['image_url'] = image_url
    new_feature.attributes['number_faces'] = face_count
    new_feature.attributes['lat'] = latitude
    new_feature.attributes['long'] = longitude
    new_feature.attributes['event_time'] = datetime.datetime.now()
    
    features_for_update.append(new_feature)
    layer.edit_features(adds = features_for_update)
    

In [ ]:
# Creates the intial node for Bad Guy, cameras, and associated data

# Function to watch a folder and detect new images on a 1 second refresh interval
before = dict ([(f, None) for f in os.listdir (img_Locs)])

count = 0
errors = 0

while True:
    count +=1
    time.sleep(delayTime)
    
    # Compares the folder contents after the sleep to what existed beforehand, and makes a list of adds and removes
    after = dict ([(f, None) for f in os.listdir (img_Locs)])
    added = [f for f in after if not f in before]
    removed = [f for f in before if not f in after]

    for z in added:
        if z.endswith('.csv'):
            added.remove(z)

    if added: print("Added: ", ", ".join (added))
    if removed: print("Removed: ", ", ".join (removed))
    before = after
    
    for img in added:
        
        if img.endswith('.jpg'):

            image, image_url = process_image(img)

            csv_file = os.path.splitext(img)[0] + ".csv"
            csv_path = os.path.join(img_Locs, csv_file)
            if os.path.exists(csv_path):
                fName, lat, long, fTime = get_spatial_info(csv_path)

            # Posts the image to the Microsoft Cognitive Services API, returns result
            mcs_response = requests.post(face_api_url, params=mcs_params, headers=mcs_headers, json={"url": image_url})

            # Return response from the Microsoft Cognitive Services API coded as JSON
            faces = mcs_response.json()

            #print(faces)
            if len(faces) > 0:
                if 'error' in faces[0].keys():
                    print(faces['error']['code'])
                    print(faces['error']['message'])
                    break

                else:
                    
                    out_fig = os.path.splitext(img)[0] + 'MarkUp.jpg'
                    image_url = web_markUp + out_fig

                    plt.figure(figsize=(8,8))
                    ax = plt.imshow(image, alpha=0.6)
                    plt.axis('off')
                    
                    face_count = 0
                    for face in faces:
                        face_count +=1
                        fr = face["faceRectangle"]
                        fa = face["faceAttributes"]
                        origin = (fr["left"], fr["top"])
                        pat = patches.Rectangle(origin, fr["width"], fr["height"], fill=False, linewidth=2, color='b')
                        ax.axes.add_patch(pat)
                        
                        plt.text(origin[0], origin[1], "%s, %d"%(fa["gender"].capitalize(), fa["age"]), \
                                fontsize=20, weight="bold", va="bottom")
                        
                        update_json(face, img, image_url, lat, long, face_count)
                        
                        face_data = dict(face)
                        faces_insert = faces_collection.insert_one(face_data).inserted_id
                        
                    outFig = os.path.splitext(img)[0] + 'MarkUp.jpg'
                    figName = os.path.join(markUp, outFig)
                    plt.savefig(figName, orientation='portrait', bbox_inches="tight", )
                    
                    geoevent_json = {}                  

                    update_json(geoevent_json, img, image_url, lat, long, face_count)
                    
                    #update_fc(gis, cameras_lyr, lat, long, img, image_url, face_count)
                        
                    plt.gcf().clear()
                    
                    responseFile =  os.path.join(responseFP,img + ".json")

                    with open(responseFile, 'w') as outfile:
                        json.dump(faces, outfile)
                        
                    geoevent_json_file =os.path.join(geoevent_json_path, img + ".json")
                    
                    with open(geoevent_json_file, 'w') as ge_file:
                        json.dump(geoevent_json, ge_file)

                    print(geoevent_json)
                    
            else:
                print("No faces detected...")
        
        if count % 100 == 0:
            print(count)

Added:  tron_1543431968.jpg
['tron', '38.92866493932333', '-77.24784393497724', '1543431823481']
{'image_name': 'tron_1543431968.jpg', 'image_url': 'http://40.76.87.212/camera/Image_MarkUp/tron_1543431968MarkUp.jpg', 'lat': '38.92866493932333', 'long': '-77.24784393497724', 'number_faces': 1}
Added:  358034087343775_1543432006.jpg
['358034087343775', '38.9564453', '-77.3804874', '1543432005']
No faces detected...
Added:  358034087343775_1543432066.jpg
['358034087343775', '38.9564629', '-77.3805193', '1543432065']
No faces detected...
Added:  358034087343775_1543432127.jpg
['358034087343775', '38.9564483', '-77.3804963', '1543432125']
No faces detected...
Added:  358034087343775_1543432186.jpg
['358034087343775', '38.9564745', '-77.3804812', '1543432185']
No faces detected...
Added:  358034087343775_1543432246.jpg
['358034087343775', '38.9564562', '-77.3804942', '1543432245']
No faces detected...
Added:  358034087343775_1543432306.jpg
['358034087343775', '38.9564621', '-77.3805197', '15

C:\Users\dif_user\Anaconda3\lib\site-packages\matplotlib\pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


{'image_name': '358034087343775_1543434406.jpg', 'image_url': 'http://40.76.87.212/camera/Image_MarkUp/358034087343775_1543434406MarkUp.jpg', 'lat': '38.9564848', 'long': '-77.380509', 'number_faces': 1}
Added:  358034087343775_1543434466.jpg
['358034087343775', '38.9564863', '-77.3805112', '1543434465']
No faces detected...
Added:  358034087343775_1543434526.jpg
['358034087343775', '38.956487', '-77.3805129', '1543434525']
{'image_name': '358034087343775_1543434526.jpg', 'image_url': 'http://40.76.87.212/camera/Image_MarkUp/358034087343775_1543434526MarkUp.jpg', 'lat': '38.956487', 'long': '-77.3805129', 'number_faces': 1}
Added:  358034087343775_1543434586.jpg
['358034087343775', '38.9564869', '-77.3805127', '1543434585']
{'image_name': '358034087343775_1543434586.jpg', 'image_url': 'http://40.76.87.212/camera/Image_MarkUp/358034087343775_1543434586MarkUp.jpg', 'lat': '38.9564869', 'long': '-77.3805127', 'number_faces': 1}
Added:  358034087343775_1543434646.jpg
['358034087343775', '3

In [ ]:
print("All Done!")